In [183]:
pip install ucimlrepo

In [184]:
import pandas as pd
import re
from collections import Counter

In [185]:
# Functions used to convert data

def yesNoBinaryConversion(input):
    
    input.replace('yes', 1, inplace = True)
    input.replace('no', 0, inplace = True)
    
def weightReplacement(input):
        
    replace_dicitonary = {r"\binsufficient": '0',
                        r"\bnormal": '1',
                       r"\boverweight": '2',
                        r"\bobesity": '3'}
    for index, value in enumerate(input):
        for key, item in replace_dicitonary.items():                 
            find_ = re.findall(key,value,re.I)
            if find_:
                input.replace(value, item, inplace=True)
                
def integerBuckets(input):
    for index, value in enumerate(input):
        if value > 3.5:
            input.replace(value, 4, inplace = True)
        elif value > 2.5:
            input.replace(value, 3, inplace = True)
        elif value > 1.5:
            input.replace(value, 2, inplace = True)
        else:
            input.replace(value, 1, inplace = True)

def categoricalBuckets(input):
    for index, value in enumerate(input):
        if value > 2.5:
            input.replace(value, 'Always', inplace = True)
        elif value > 1.5:
            input.replace(value, 'Frequently', inplace = True)
        else:
            input.replace(value, 'Sometimes', inplace = True)

In [186]:
from ucimlrepo import fetch_ucirepo 
  
# fetch dataset 
estimation_of_obesity_levels_based_on_eating_habits_and_physical_condition = fetch_ucirepo(id=544) 
  
# data (as pandas dataframes) 
feature_df = estimation_of_obesity_levels_based_on_eating_habits_and_physical_condition.data.features 
y = estimation_of_obesity_levels_based_on_eating_habits_and_physical_condition.data.targets 

In [187]:
# View metadata and variable types
print(estimation_of_obesity_levels_based_on_eating_habits_and_physical_condition.variables[['name', 'type', 'description']])

                              name         type  \
0                           Gender  Categorical   
1                              Age   Continuous   
2                           Height   Continuous   
3                           Weight   Continuous   
4   family_history_with_overweight       Binary   
5                             FAVC       Binary   
6                             FCVC      Integer   
7                              NCP   Continuous   
8                             CAEC  Categorical   
9                            SMOKE       Binary   
10                            CH2O   Continuous   
11                             SCC       Binary   
12                             FAF   Continuous   
13                             TUE      Integer   
14                            CALC  Categorical   
15                          MTRANS  Categorical   
16                      NObeyesdad  Categorical   

                                          description  
0                        

# Processing Feature Data

In [188]:
# View df to see how data is stored. APply appropriate data conversions.
feature_df.head()

,Gender,Age,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,SMOKE,CH2O,SCC,FAF,TUE,CALC,MTRANS
0,Female,21.0,1.62,64.0,yes,no,2.0,3.0,Sometimes,no,2.0,no,0.0,1.0,no,Public_Transportation
1,Female,21.0,1.52,56.0,yes,no,3.0,3.0,Sometimes,yes,3.0,yes,3.0,0.0,Sometimes,Public_Transportation
2,Male,23.0,1.80,77.0,yes,no,2.0,3.0,Sometimes,no,2.0,no,2.0,1.0,Frequently,Public_Transportation
3,Male,27.0,1.80,87.0,no,no,3.0,3.0,Sometimes,no,2.0,no,2.0,0.0,Frequently,Walking
4,Male,22.0,1.78,89.8,no,no,2.0,1.0,Sometimes,no,2.0,no,0.0,0.0,Sometimes,Public_Transportation


In [189]:
# Replace binary value types with 1 for yes, 0 for no for all binary type columns

binary_list = []

for index, row in estimation_of_obesity_levels_based_on_eating_habits_and_physical_condition['variables'].iterrows():    
    
    
    if row['type'] == 'Binary':
        binary_list.append(row['name'])
        
for i in binary_list:
    yesNoBinaryConversion(feature_df[i])

c:\Users\anups\anaconda3\envs\dev0\lib\site-packages\pandas\core\generic.py:6619: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)


In [190]:
# Create a column which represents a known ratio of weight to height, i.e. BMI.
feature_df['BMI'] = round(feature_df['Weight'] / feature_df['Height']**2, 3)

In [191]:
# Place the FCVC feature into buckets. FCVC is the question of regular fruit/veg consumption. 1 - Always, 2 - Frequently, 3 - Never
categoricalBuckets(feature_df['FCVC'])

print(Counter(feature_df['FCVC']))

Counter({'Frequently': 1013, 'Always': 996, 'Sometimes': 102})


In [192]:
# Place the NCP feature into buckets. NCP is the question of how many meals per day.

integerBuckets(feature_df['NCP'])

print(Counter(feature_df['NCP']))

Counter({3.0: 1470, 1.0: 316, 2.0: 176, 4.0: 149})


In [193]:
feature_df.drop(columns=['Height', 'Weight'], inplace = True)

In [194]:
feature_copy = pd.get_dummies(feature_df)

In [195]:
feature_copy.head()

,Age,family_history_with_overweight,FAVC,NCP,SMOKE,CH2O,SCC,FAF,TUE,BMI,...,CAEC_no,CALC_Always,CALC_Frequently,CALC_Sometimes,CALC_no,MTRANS_Automobile,MTRANS_Bike,MTRANS_Motorbike,MTRANS_Public_Transportation,MTRANS_Walking
0,21.0,1,0,3.0,0,2.0,0,0.0,1.0,24.387,...,0,0,0,0,1,0,0,0,1,0
1,21.0,1,0,3.0,1,3.0,1,3.0,0.0,24.238,...,0,0,0,1,0,0,0,0,1,0
2,23.0,1,0,3.0,0,2.0,0,2.0,1.0,23.765,...,0,0,1,0,0,0,0,0,1,0
3,27.0,0,0,3.0,0,2.0,0,2.0,0.0,26.852,...,0,0,1,0,0,0,0,0,0,1
4,22.0,0,0,1.0,0,2.0,0,0.0,0.0,28.342,...,0,0,0,1,0,0,0,0,1,0


In [196]:
X = feature_copy.copy()

#### Final X (feature) dataframe

In [197]:
X.head()

,Age,family_history_with_overweight,FAVC,NCP,SMOKE,CH2O,SCC,FAF,TUE,BMI,...,CAEC_no,CALC_Always,CALC_Frequently,CALC_Sometimes,CALC_no,MTRANS_Automobile,MTRANS_Bike,MTRANS_Motorbike,MTRANS_Public_Transportation,MTRANS_Walking
0,21.0,1,0,3.0,0,2.0,0,0.0,1.0,24.387,...,0,0,0,0,1,0,0,0,1,0
1,21.0,1,0,3.0,1,3.0,1,3.0,0.0,24.238,...,0,0,0,1,0,0,0,0,1,0
2,23.0,1,0,3.0,0,2.0,0,2.0,1.0,23.765,...,0,0,1,0,0,0,0,0,1,0
3,27.0,0,0,3.0,0,2.0,0,2.0,0.0,26.852,...,0,0,1,0,0,0,0,0,0,1
4,22.0,0,0,1.0,0,2.0,0,0.0,0.0,28.342,...,0,0,0,1,0,0,0,0,1,0


In [198]:
# Check to see if all are int/float (objective)

X.dtypes

Age                               float64
family_history_with_overweight      int64
FAVC                                int64
NCP                               float64
SMOKE                               int64
CH2O                              float64
SCC                                 int64
FAF                               float64
TUE                               float64
BMI                               float64
Gender_Female                       uint8
Gender_Male                         uint8
FCVC_Always                         uint8
FCVC_Frequently                     uint8
FCVC_Sometimes                      uint8
CAEC_Always                         uint8
CAEC_Frequently                     uint8
CAEC_Sometimes                      uint8
CAEC_no                             uint8
CALC_Always                         uint8
CALC_Frequently                     uint8
CALC_Sometimes                      uint8
CALC_no                             uint8
MTRANS_Automobile                 

## Processing target data (y)

In [199]:
# Need to Normalise the target data
y.head()

,NObeyesdad
0,Normal_Weight
1,Normal_Weight
2,Normal_Weight
3,Overweight_Level_I
4,Overweight_Level_II


#### For the purpose of buckets, I've only used 'underweight - 0', 'normal weight - 1', 'overweight - 2', 'obese - 3'

In [200]:
# Create buckets to place the target data in, and run the function

print(y['NObeyesdad'].value_counts())

# Apply function
y.apply(weightReplacement)

#Check new buckets
print(y['NObeyesdad'])

Obesity_Type_I         351
Obesity_Type_III       324
Obesity_Type_II        297
Overweight_Level_I     290
Overweight_Level_II    290
Normal_Weight          287
Insufficient_Weight    272
Name: NObeyesdad, dtype: int64
0       1
1       1
2       1
3       2
4       2
       ..
2106    3
2107    3
2108    3
2109    3
2110    3
Name: NObeyesdad, Length: 2111, dtype: object


#### Processed target data

In [207]:
y.head()

,NObeyesdad
0,1
1,1
2,1
3,2
4,2
